# Louvain and Leiden Community Detection


In this notebook, we will use cuGraph to identify the cluster in a test graph using both the Louvain and Leiden algorithms  

| Author Credit |    Date    |  Update          | cuGraph Version |  Test Hardware              |
| --------------|------------|------------------|-----------------|-----------------------------|
| Brad Rees     | 08/01/2019 | created          | 0.6             |                             |
|               | 08/16/2020 | updated          | 0.14            | GV100, CUDA 10.2            |
|               | 08/05/2021 | tested / updated | 21.10 nightly   | RTX 3090 CUDA 11.4          |
| Don Acosta    | 07/11/2022 | tested / updated | 22.08 nightly   | DGX Tesla V100 CUDA 11.5    |
| Ralph Liu    | 07/26/2022 | updated | 22.08 nightly   | DGX Tesla V100 CUDA 11.5    |



## Louvain Introduction

The Louvain method of community detection is a greedy hierarchical clustering algorithm which seeks to optimize Modularity as it progresses. Louvain starts with each vertex in its own clusters and iteratively merges groups using graph contraction.  

For a detailed description of the algorithm see: https://en.wikipedia.org/wiki/Louvain_Modularity

It takes as input a cugraph.Graph object and returns as output a 
cudf.Dataframe object with the id and assigned partition for each 
vertex as well as the final modularity score

To compute the Louvain cluster in cuGraph use: <br>
   __df, mod  = cugraph.louvain(G, max_iter = 100)__
   
  
    Parameters
    ----------
    input_graph : cugraph.Graph
        cuGraph graph descriptor, should contain the connectivity information
        as an edge list. The adjacency list will be computed if not already present.
        The graph should be undirected where an undirected edge is represented by a
        directed edge in both direction.

    max_iter : integer
        This controls the maximum number of levels/iterations of the Louvain
        algorithm. When specified the algorithm will terminate after no more
        than the specified number of iterations. No error occurs when the
        algorithm terminates early in this manner.

    Returns
    -------
    parts : cudf.DataFrame
        A GPU data frame of size V containing two columns the vertex id and the
        partition id it is assigned to.
        
            df[‘vertex’] cudf.Series
                Contains the vertex identifiers
            df[‘partition’] cudf.Series
                Contains the partition assigned to the vertices
        
        
    modularity_score : float
        a floating point number containing the modularity score of the
        partitioning.
        
    All vertices with the same partition ID are in the same cluster
        


#### Note
Parallel Louvain produces different modularity scores than serial Louvain.

Serial Louvain is a greedy method that iterates over the vertices in numerical order and
evaluates whether to move a vertex into a different cluster and then moves it if
modularity is increased.  After considering all vertices in serial it computes a new
modularity score and if better than the last clustering it will repeat.  If vertex i is
moved to a new cluster, and vertex j is a neighbor of vertex i and is processed after
vertex i, then vertex j will compute its delta modularity based on the vertex i's new
cluster assignment.

Parallel Louvain computes delta modularity for all vertex/cluster combinations and chooses
the best change for ALL vertices in parallel.  This means that vertex i and vertex j being
neighbors (as above), both will see the old cluster assignment for that vertex.  This will result
in different decisions being made between the parallel and serial versions.  Because Louvain
is a greedy method, those decisions might lead the algorithms to converge on different local
maximums.

A complete technical write-up is being produced and will be linked here when available.

### References
* Blondel, V. D., Guillaume, J.-L., Lambiotte, R., and Lefebvre, E. Fast unfolding of communities in large networks. Journal of statistical mechanics: theory and experiment 2008, 10 (2008), P10008.

## Leiden Introduction
Leiden tries to improve upon Louvain by only moving vertices that result in higher quality clusters.

Leiden has the same set of arguments and return type as Louvain

### References
* Traag, V. A., Waltman, L., & van Eck, N. J. (2019). From Louvain to Leiden: guaranteeing well-connected communities. Scientific reports, 9(1), 5233.
  doi: 10.1038/s41598-019-41695-z

#### Some notes about vertex IDs...

* cuGraph will automatically renumber graphs to an internal format consisting of a contiguous series of integers starting from 0, and convert back to the original IDs when returning data to the caller. If the vertex IDs of the data are already a contiguous series of integers starting from 0, the auto-renumbering step can be skipped for faster graph creation times.
  * To skip auto-renumbering, set the `renumber` boolean arg to `False` when calling the appropriate graph creation API (eg. `G.from_cudf_edgelist(gdf_r, source='src', destination='dst', renumber=False)`).
  * For more advanced renumbering support, see the examples in `structure/renumber.ipynb` and `structure/renumber-2.ipynb`


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


<img src="../../img/zachary_graph_clusters.png" width="35%"/>

Because the test data has vertex IDs starting at 1, the auto-renumber feature of cuGraph (mentioned above) will be used so the starting vertex ID is zero for maximum efficiency. The resulting data will then be auto-unrenumbered, making the entire renumbering process transparent to users.


### Prep

In [ ]:
# Import needed libraries
import cugraph
import cudf

# Import a built-in dataset
from cugraph.experimental.datasets import karate

## Create an Edgelist

In [ ]:
# You can also just get the edgelist
gdf = karate.get_edgelist(fetch=True)

# The algorithm also requires that there are vertex weights.  Just use 1.0 
gdf["data"] = 1.0

In [ ]:
# just for fun, let's look at the data types in the dataframe
gdf.dtypes

In [ ]:
# create a Graph - since the data does not start at '0', use the auto-renumbering feature
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst', edge_attr='data', renumber=True)

In [ ]:
# Call Louvain on the graph
df, mod = cugraph.louvain(G) 

In [ ]:
# Call Leiden on the graph
df_l, mod_l = cugraph.leiden(G) 

In [ ]:
# Print the modularity score
print('Louvain Modularity was {}'.format(mod))
print()

In [ ]:
# Print the modularity score
print('Leiden Modularity was {}'.format(mod_l))
print()

In [ ]:
# How many Louvain partitions where found
part_ids = df["partition"].unique()

In [ ]:
print("Louvain found " + str(len(part_ids)) + " partitions")

In [ ]:
# print the clusters.  
for p in range(len(part_ids)):
    part = []
    for i in range(len(df)):
        if (df['partition'].iloc[i] == p):
            part.append(df['vertex'].iloc[i] )
    print("Partition " + str(p) + ":")
    print(part)


In [ ]:
# How many Leiden partitions were found
part_ids_l = df_l["partition"].unique()
print("Leiden found " + str(len(part_ids_l)) + " partitions")

In [ ]:
# print the clusters.  
for p in range(len(part_ids_l)):
    part = []
    for i in range(len(df)):
        if (df['partition'].iloc[i] == p):
            part.append(df['vertex'].iloc[i] )
    print("Partition " + str(p) + ":")
    print(part)

___
Copyright (c) 2019-2022, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___